In [42]:
import yfinance as yf
import pandas as pd

# List of tickers
tickers = ['SPY', 'XLK', 'XLY', 'XLF', 'XLV']

atm_data = []

for ticker in tickers:
    stock = yf.Ticker(ticker)
    expirations = stock.options
    if not expirations:
        print(f"No expirations found for {ticker}")
        continue
    exp_date = expirations[0]  # soonest expiry

    # Get option chain for that date
    opt_chain = stock.option_chain(exp_date)
    calls = opt_chain.calls
    puts = opt_chain.puts

    # Get current price
    try:
        current_price = stock.history(period="1d")['Close'].iloc[-1]
    except Exception:
        current_price = None

    # Find ATM call (closest strike to current price)
    if current_price is not None and not calls.empty:
        calls['abs_diff'] = (calls['strike'] - current_price).abs()
        atm_call = calls.loc[calls['abs_diff'].idxmin()]
    else:
        atm_call = None

    # Find ATM put (closest strike to current price)
    if current_price is not None and not puts.empty:
        puts['abs_diff'] = (puts['strike'] - current_price).abs()
        atm_put = puts.loc[puts['abs_diff'].idxmin()]
    else:
        atm_put = None

    atm_data.append({
        'ticker': ticker,
        'exp_date': exp_date,
        'current_price': current_price,
        'atm_call_strike': atm_call['strike'] if atm_call is not None else None,
        'atm_call_bid': atm_call['bid'] if atm_call is not None else None,
        'atm_call_ask': atm_call['ask'] if atm_call is not None else None,
        'atm_call_iv': atm_call['impliedVolatility'] if atm_call is not None and 'impliedVolatility' in atm_call else None,
        'atm_put_strike': atm_put['strike'] if atm_put is not None else None,
        'atm_put_bid': atm_put['bid'] if atm_put is not None else None,
        'atm_put_ask': atm_put['ask'] if atm_put is not None else None,
        'atm_put_iv': atm_put['impliedVolatility'] if atm_put is not None and 'impliedVolatility' in atm_put else None,
    })

# Compile into DataFrame
atm_df = pd.DataFrame(atm_data)
atm_df

,ticker,exp_date,current_price,atm_call_strike,atm_call_bid,atm_call_ask,atm_call_iv,atm_put_strike,atm_put_bid,atm_put_ask,atm_put_iv
0,SPY,2025-08-11,637.479980,637.0,0.0,0.0,0.00001,637.0,0.0,0.0,0.003916
1,XLK,2025-08-15,265.829987,265.0,0.0,0.0,0.00001,265.0,0.0,0.0,0.007822
2,XLY,2025-08-15,225.250000,225.0,0.0,0.0,0.00001,225.0,0.0,0.0,0.003916
3,XLF,2025-08-15,52.014999,52.0,0.0,0.0,0.00001,52.0,0.0,0.0,0.001963
4,XLV,2025-08-15,131.089996,131.0,0.0,0.0,0.00001,131.0,0.0,0.0,0.001963


In [43]:
sector_etf_map = {
    'realestate': 'XLRE',
    'consumer_cyclical': 'XLY',
    'basic_materials': 'XLB',
    'consumer_defensive': 'XLP',
    'technology': 'XLK',
    'communication_services': 'XLC',
    'financial_services': 'XLF',
    'utilities': 'XLU',
    'industrials': 'XLI',
    'energy': 'XLE',
    'healthcare': 'XLV'
}


In [44]:
import yfinance as yf
import pandas as pd

# Get SPY ETF info
spy = yf.Ticker("SPY")

# Pull sector weightings (if available)
sector_weights = spy.funds_data.sector_weightings
# Print sector breakdown
print(sector_weights)



{'realestate': 0.0199, 'consumer_cyclical': 0.10560001, 'basic_materials': 0.0161, 'consumer_defensive': 0.0523, 'technology': 0.3529, 'communication_services': 0.0986, 'financial_services': 0.1337, 'utilities': 0.024500001, 'industrials': 0.0784, 'energy': 0.03, 'healthcare': 0.088}


In [45]:
sorted_sectors = sorted(sector_weights.items(), key=lambda x: x[1], reverse=True)
top4 = sorted_sectors[:4]
total_top4 = sum(w for _, w in top4)
# Scale to 100%
scaled_top4 = {sector: weight / total_top4 * 100 for sector, weight in top4}
# Add SPY as 100%
scaled_top4['SPY'] = 100.0
# Get last close price for each sector ETF and SPY
etf_map = {
    'technology': 'XLK',
    'financial_services': 'XLF',
    'consumer_cyclical': 'XLY',
    'healthcare': 'XLV',
    'communication_services': 'XLC',
    'realestate': 'XLRE',
    'basic_materials': 'XLB',
    'consumer_defensive': 'XLP',
    'utilities': 'XLU',
    'industrials': 'XLI',
    'energy': 'XLE',
    'SPY': 'SPY'
}
data = []
for sector, weight in scaled_top4.items():
    ticker = etf_map.get(sector, None)
    if ticker:
        try:
            close = yf.Ticker(ticker).history(period="1d")['Close'].iloc[-1]
        except Exception:
            close = None
    else:
        close = None
    data.append({'sector': sector, 'ticker': ticker, 'weight': weight, 'last_close': close})
df = pd.DataFrame(data)

df['vector'] = df['weight'].values / df['last_close'].values
print(df)

                   sector ticker      weight  last_close    vector
0              technology    XLK   51.085697  265.829987  0.192174
1      financial_services    XLF   19.354371   52.014999  0.372092
2       consumer_cyclical    XLY   15.286625  225.250000  0.067865
3  communication_services    XLC   14.273306  108.290001  0.131806
4                     SPY    SPY  100.000000  637.479980  0.156868


In [46]:
import numpy as np

# Try a range of scaling factors and pick the one that minimizes the sum of fractional parts
scaling_factors = np.arange(1, 101, 0.1)
best_factor = None
best_score = float('inf')

for factor in scaling_factors:
    scaled = df['vector'].values * factor
    score = np.sum(np.abs(scaled - np.round(scaled)))  # sum of fractional parts
    if score < best_score:
        best_score = score
        best_factor = factor

print(f"Best scaling factor: {best_factor:.2f}")
print("Scaled values (rounded):")
print(np.round(df['weight'].values / df['last_close'].values * best_factor))
df['vector'] = df['weight'].values / df['last_close'].values * best_factor

Best scaling factor: 83.30
Scaled values (rounded):
[16. 31.  6. 11. 13.]


In [47]:
# Calculate the cost of ATM straddles for each ticker and scale by vector

# Assume you already have atm_df (with ATM call/put bid/ask) and df (with vector column)
# We'll use the ask price for both call and put for a conservative estimate

def calc_straddle_cost(row):
    # Find the corresponding ATM row for this ticker
    atm_row = atm_df[atm_df['ticker'] == row['ticker']]
    if atm_row.empty:
        return None
    atm_row = atm_row.iloc[0]
    call_ask = atm_row['atm_call_ask']
    put_ask = atm_row['atm_put_ask']
    if pd.isna(call_ask) or pd.isna(put_ask):
        return None
    # Option contracts are for 100 shares
    straddle_cost = (call_ask + put_ask) * 100
    return straddle_cost * row['vector']

df['atm_straddle_cost'] = df.apply(calc_straddle_cost, axis=1)
print(df[['ticker', 'vector', 'atm_straddle_cost']])
print(f"Total scaled straddle cost: {df['atm_straddle_cost'].sum():.2f}")

  ticker     vector  atm_straddle_cost
0    XLK  16.008121                0.0
1    XLF  30.995274                0.0
2    XLY   5.653167                0.0
3    XLC  10.979466                NaN
4    SPY  13.067077                0.0
Total scaled straddle cost: 0.00
